# Day 1

In [8]:
from itertools import pairwise, tee
from pathlib import Path

INPUTS = Path("aoc2021_inputs")

# With lazy iterator:
count = 0
with open(INPUTS / "day1.txt", "r") as f:
    for prev_, next_ in pairwise(map(int, f)):
        count += next_ > prev_
assert count == 1292

def triplewise(it):
    a, b, c = tee(it, 3)
    next(b); next(c); next(c)
    yield from zip(a, b, c)
    
def nwise(it, n):
    its = tee(it, n)
    for i, it in enumerate(its):
        for _ in range(i):
            next(it)
    yield from zip(*its)
    
def triplewise_(it):
    yield from nwise(it, 3)

count = 0
with open(INPUTS / "day1.txt", "r") as f:
    sum_prev = float("+inf")
    for v1, v2, v3 in triplewise_(map(int, f)):
        sum_ = v1 + v2 + v3
        count += sum_ > sum_prev
        sum_prev = sum_
assert count == 1262

# Day 2

In [2]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day2.txt")

with open(INPUT_PATH, "r") as f:
    moves = f.readlines()

h, v = 0, 0
for line in moves:
    move, size = line.split()

    if move == "forward":
        h += int(size)
    elif move == "up":
        v -= int(size)
    else:
        v += int(size)
        
print(h, v, h * v)
assert h * v == 1727835

1905 907 1727835


In [3]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day2.txt")

with open(INPUT_PATH, "r") as f:
    moves = f.readlines()
    
horiz, depth, aim = 0, 0, 0
for line in moves:
    move, size = line.split()
    size = int(size)
    
    if move == "forward":
        horiz += size
        depth += size * aim
    elif move == "down":
        aim += size
    else:
        aim -= size
        
print(horiz, depth, aim, horiz * depth)

1905 810499 907 1544000595


# Day 3

In [39]:
from collections import Counter
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day3.txt")

counter = Counter()
with open(INPUT_PATH, "r") as f:
    for line in f:
        counter += Counter(enumerate(line.strip()))

gamma, epsilon = 0, 0
total_bits = len(counter) // 2
for n in range(total_bits):
    if counter[(n, "1")] > counter[(n, "0")]:
        gamma |= 1 << (total_bits - n - 1)
    else:
        epsilon |= 1 << (total_bits - n - 1)
print(gamma * epsilon)

# --- part 2 ---

with open(INPUT_PATH, "r") as f:
    lines = f.readlines()

under_consideration = lines
prefix = ""
while len(under_consideration) > 1:
    c = Counter(line[:len(prefix) + 1] for line in under_consideration)
    c[prefix + "1"] += 0.5  # Force ending in "1" to win in case of draw.
    (prefix, count), = c.most_common(1)
    under_consideration = [line for line in under_consideration if line.startswith(prefix)]
oxygen = int(under_consideration[0], 2)
print(oxygen)

under_consideration = lines
prefix = ""
while len(under_consideration) > 1:
    c = Counter(line[:len(prefix) + 1] for line in under_consideration)
    c[prefix + "0"] -= 0.5  # Force ending in "0" to lose in case of draw
    _, (prefix, count) = c.most_common(2)
    under_consideration = [line for line in under_consideration if line.startswith(prefix)]
co2 = int(under_consideration[0], 2)
print(co2)

print(oxygen * co2)

749376
3871
613
2372923


# Day 4

In [3]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day4.txt")

with open(INPUT_PATH, "r") as f:
    nums = next(f).strip().split(",")  # Read order in which numbers are drawn.
    data = f.read().strip()

# Build dictionary storing the position of each number.
num_pos = dict(zip(nums, range(len(nums))))

# `boards` is a list of pairs (dirs, nums)
# `dirs` contains a list of all the possible winning directions for a board;
# `nums` is the set of all the numbers that show up in that board.
boards = []
for subdata in data.split("\n\n"):
    # Store the rows and columns in the same list.
    directions = []
    # Set to keep track of all the numbers the card contains.
    numbers = set()
    for row in subdata.split("\n"):
        directions.append(row.split())
        # Update the numbers with the ones of the current row.
        numbers |= {*directions[-1]}
    # Add columns to the directions list.
    bs = len(directions)
    directions += [
        [directions[r][c] for r in range(bs)] for c in range(bs)
    ]

    boards.append((directions, numbers))

# Go over each board and figure out when that board is done.
done_at = [
    min(max(num_pos[n] for n in dir_) for dir_ in dirs)
    for dirs, _ in boards
]

def score_board(winning_move_idx):
    _, board_nums = boards[done_at.index(winning_move_idx)]
    unmarked_numbers = map(int, board_nums - set(nums[:winning_move_idx + 1]))
    return sum(unmarked_numbers) * int(nums[winning_move_idx])

print(score_board(min(done_at))) # part 1
print(score_board(max(done_at))) # part 2

27027
36975


# Day 5

In [39]:
from collections import Counter
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day5.txt")

def parse_line(string):
    return [list(map(int, point.split(","))) for point in string.split(" -> ")]

def sign(x):
    return (x > 0) - (x < 0)

def generate_segment(left, right):
    (xl, yl), (xr, yr) = left, right
    delta = max(abs(xl - xr), abs(yl - yr))
    dx, dy = sign(xr - xl), sign(yr - yl)
    return [(xl + dx * d, yl + dy * d) for d in range(delta + 1)]

with open(INPUT_PATH, "r") as f:
    points = [parse_line(line.strip()) for line in f]

counter = Counter()
for left, right in points:
    (xl, yl), (xr, yr) = left, right
    if xl != xr and yl != yr:
        continue
    counter += Counter(generate_segment(left, right))

print(sum(cnt > 1 for _, cnt in counter.items()))

counter = Counter()
for left, right in points:
    counter += Counter(generate_segment(left, right))

print(sum(cnt > 1 for _, cnt in counter.items()))

7142
20012


# Day 6

In [1]:
from collections import Counter
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day6.txt")

with open(INPUT_PATH, "r") as f:
    lanternfish = [int(num) for num in f.readline().split(",")]
    
population = Counter(lanternfish)
sizes = [population.total()]
for cycle in [80, 256 - 80]:
    for _ in range(cycle):
        next_gen = Counter()
        for age in range(8, 0, -1):
            next_gen[age - 1] = population[age]
        next_gen[8] = population[0]
        next_gen[6] += population[0]
        population = next_gen
        sizes.append(population.total())
    print(population.total())  # New in Python 3.10

345387
1574445493136


# Day 7

In [22]:
from math import ceil
from pathlib import Path
from statistics import mean, median

INPUT_PATH = Path("aoc2021_inputs/day7.txt")

with open(INPUT_PATH, "r") as f:
    crabs = [int(num) for num in f.readline().split(",")]

linear_alignment = round(median(crabs))
print(sum(abs(linear_alignment - crab) for crab in crabs))
quad_alignment_options = [c := ceil(mean(crabs)), c - 1]
print(min(sum(
    (d := abs(quad_alignment - crab)) * (d + 1) // 2
     for crab in crabs
) for quad_alignment in quad_alignment_options))

349812
99763899


# Day 8

In [20]:
from collections import Counter
from functools import reduce
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day8.txt")

segment_lengths = [6, 2, 5, 5, 4, 5, 6, 3, 7, 6]

def with_length(clues, l):
    return {s for s in clues if len(s) == l}

def comp(clue):
    return frozenset("abcdefg") - clue

def solve_clues(clues):
    one = with_length(clues, 2).pop()
    four = with_length(clues, 4).pop()
    seven = with_length(clues, 3).pop()
    eight = with_length(clues, 7).pop()

    zero_six_nine = with_length(clues, 6)
    six = [s for s in zero_six_nine if comp(s) < one][0]
    zero_nine = zero_six_nine - {six}
    zero = [s for s in zero_nine if comp(s) < four][0]
    nine = (zero_nine - {zero}).pop()
    
    two_three_five = with_length(clues, 5)
    three = [s for s in two_three_five if one < s][0]
    two_five = two_three_five - {three}
    five = [s for s in two_five if s < nine][0]
    two = (two_five - {five}).pop()
    return [zero, one, two, three, four, five, six, seven, eight, nine]
    

counter = Counter()
with open(INPUT_PATH, "r") as f:
    for line in f:
        _, digits = line.split(" | ")
        counter.update(map(len, digits.strip().split()))
print(sum(counter[segment_lengths[d]] for d in [1, 4, 7, 8]))

outs = []
with open(INPUT_PATH, "r") as f:
    for line in f:
        clue_strings, digits = line.split(" | ")
        clues = [frozenset(string) for string in clue_strings.split()]
        solutions = solve_clues(clues)
        digits = [solutions.index(frozenset(string)) for string in digits.strip().split()]
        outs.append(reduce(lambda l, r: 10 * l + r, digits))
print(sum(outs))

264
1063760


# Day 9

In [15]:
from math import prod
from pathlib import Path

def neighbouring_positions(matrix, r, c):
    return {
        (r_, c_) for r_, c_ in [(r - 1, c), (r, c - 1), (r + 1, c), (r, c + 1)]
        if 0 <= r_ < len(matrix) and 0 <= c_ < len(matrix[0])
    }

def get_neighbours(matrix, r, c):
    to_try = neighbouring_positions(matrix, r, c)
    return {matrix[r_][c_] for r_, c_ in to_try}

def find_basin(matrix, r, c):
    to_visit, visited, basin = {(r, c)}, set(), set()
    while to_visit:
        r_, c_ = to_visit.pop()
        if matrix[r_][c_] == 9:
            continue
        basin.add((r_, c_))
        next_neighbours = neighbouring_positions(matrix, r_, c_)
        to_visit.update(next_neighbours - visited)
        visited.update(next_neighbours)
    return basin

INPUT_PATH = Path("aoc2021_inputs/day9.txt")

with open(INPUT_PATH, "r") as f:
    matrix = [
        [int(num) for num in line.strip()]
        for line in f.readlines()
    ]

risk = 0
low_point_locations = []
for r, row in enumerate(matrix):
    for c, val in enumerate(row):
        neighbs = get_neighbours(matrix, r, c)
        if min(neighbs) > val:
            risk += val + 1
            low_point_locations.append((r, c))
print(risk)

basin_sizes = []
for low_point in low_point_locations:
    basin = find_basin(matrix, *low_point)
    basin_sizes.append(len(basin))
print(prod(sorted(basin_sizes)[-3:]))

502
1330560


# Day 10

In [8]:
from functools import reduce
from pathlib import Path
from statistics import median

OPENING = {"(", "[", "{", "<"}  # Set with opening characters.
CLOSING = {")": "(", "]": "[", "}": "{", ">": "<"}  # Dict with opening character for each closing.
CLOSE_WITH = {v: k for k, v in CLOSING.items()}     # Dict with closing character for each opening.
CORRUPTED_SCORES = {")": 3, "]": 57, "}": 1197, ">": 25137}  # Scores.
COMPLETION_SCORES = {")": 1, "]": 2, "}": 3, ">": 4}

def from_digits(digits, base):
    return reduce(lambda l, r: l * base + r, digits, 0)

INPUT_PATH = Path("aoc2021_inputs/day10.txt")

with open(INPUT_PATH, "r") as f:
    lines = f.readlines()

corrupted_scores = 0
completion_scores = []
for line in lines:
    stack = []
    for char in line:
        if char in OPENING:
            stack.append(char)
        elif char in CLOSING:
            if stack[-1] == CLOSING[char]:
                stack.pop()
            else:
                corrupted_scores += CORRUPTED_SCORES[char]
                break
    else:
        digits = [COMPLETION_SCORES[CLOSE_WITH[char]] for char in reversed(stack)]
        completion_scores.append(from_digits(digits, 5))
print(corrupted_scores)
print(median(completion_scores))

370407
3249889609


# Day 11

In [13]:
from collections import defaultdict
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day11.txt")

grid = defaultdict(lambda: float("-inf"))
with open(INPUT_PATH, "r") as f:
    grid |= {
        (r, c): int(val)
        for r, line in enumerate(f) for c, val in enumerate(line.strip())
    }

def neighbouring_positions(r, c):
    return [
        (r + 1, c), (r + 1, c + 1), (r, c + 1), (r - 1, c + 1),
        (r - 1, c), (r - 1, c - 1), (r, c - 1), (r + 1, c - 1),
    ]

flashes, flashed, step = 0, set(), 0
while len(flashed) < 100:
    step += 1
    flashed = set()
    to_increment = [(r, c) for r in range(10) for c in range(10)]
    while to_increment:
        r, c = to_increment.pop()
        if not ((0 <= r < 10) and (0 <= c < 10)):
            continue
        grid[r, c] += 1
        if grid[r, c] > 9 and (r, c) not in flashed:
            flashes += 1
            flashed.add((r, c))
            to_increment.extend(neighbouring_positions(r, c))
    for r in range(10):
        for c in range(10):
            grid[r, c] = 0 if grid[r, c] > 9 else grid[r, c]
    if step == 100:
        print(flashes)
print(step)

1585
382


# Day 12

In [15]:
from collections import defaultdict
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day12.txt")

def count_paths(graph, path_built, can_revisit=True):
    if path_built and path_built[-1] == "end":
        return 1

    return sum(
        count_paths(
            graph,
            path_built + [next_stop],
            can_revisit and not (next_stop.islower() and next_stop in path_built),
        )
        for next_stop in graph[path_built[-1]]
        if next_stop not in path_built or next_stop.isupper() or can_revisit
    )

with open(INPUT_PATH, "r") as f:
    lines = f.readlines()

graph = defaultdict(list)
for line in lines:
    pt1, pt2 = line.strip().split("-")
    graph[pt1].append(pt2)
    graph[pt2].append(pt1)
# Make sure no cave points back at the “start” cave.
for connections in graph.values():
    try:
        connections.remove("start")
    except ValueError:
        pass

print(count_paths(graph, ["start"], False))
print(count_paths(graph, ["start"], True))

3410
98796


# Day 13

In [2]:
from itertools import chain
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day13.txt")

with open(INPUT_PATH, "r") as f:
    contents = f.read()

def fold(points, fold_point):
    xf, yf = fold_point
    return {
        (x if x < xf else 2 * xf - x, y if y < yf else 2 * yf - y)
        for x, y in points
    }

point_data, fold_data = contents.split("\n\n")
points = {tuple(map(int, line.split(","))) for line in point_data.splitlines()}
folds = fold_data.splitlines()

MAX_COORD = 1 + max(chain.from_iterable(points))

for fold_string in folds:
    coord, value = fold_string.removeprefix("fold along ").split("=")
    fold_point = (
        int(value) if coord == "x" else MAX_COORD,
        int(value) if coord == "y" else MAX_COORD,
    )
    points = fold(points, fold_point)

width = max(x for x, _ in points) + 1
height = max(y for _, y in points) + 1
result = [[" "] * width for _ in range(height)]
for x, y in points:
    result[y][x] = "\u2588"
print("\n".join("".join(line) for line in result))

████  ██  █  █  ██  █  █ ███  ████  ██ 
█    █  █ █ █  █  █ █ █  █  █    █ █  █
███  █    ██   █    ██   ███    █  █   
█    █ ██ █ █  █    █ █  █  █  █   █ ██
█    █  █ █ █  █  █ █ █  █  █ █    █  █
█     ███ █  █  ██  █  █ ███  ████  ███


# Day 14

In [28]:
from collections import Counter
from itertools import chain
from math import ceil
from pathlib import Path

try:
    from itertools import pairwise  # Python 3.10
except ImportError:
    from itertools import tee
    def pairwise(it):
        it1, it2 = tee(it, 2)
        next(it2)
        yield from zip(it1, it2)

INPUT_PATH = Path("aoc2021_inputs/day14.txt")

with open(INPUT_PATH, "r") as f:
    polymer = f.readline().strip()
    f.readline()  # empty line
    rules = dict(line.strip().split(" -> ") for line in f)

counter = Counter("".join(item) for item in pairwise(polymer))

for _ in range(40):
    new_counter = Counter()
    for (char1, char2), count in counter.items():
        target = rules[char1 + char2]
        new_counter[char1 + target] += count
        new_counter[target + char2] += count
    counter = new_counter

element_counter = Counter()
for (char1, char2), count in counter.items():
    element_counter[char1] += count
    element_counter[char2] += count
print(ceil(max(element_counter.values()) / 2) - ceil(min(element_counter.values()) / 2))

3015383850689


In [26]:
polymer

'FNFPPNKPPHSOKFFHOFOC'